In [18]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [19]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2012t+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797868,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797869,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797870,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797871,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [20]:
data = pd.read_csv('общие_данные_2012t+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]


protests = data[data['Event Text'].isin(protest_events)].copy()
protests_0910 = protests[protests['year'].isin([2011, 2012, 2013])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('oktmo')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['oktmo']).reset_index(drop=True)



protests.to_csv('протесты_2012t+.csv', index=False)
protests1.to_csv('протесты2012_2012t+.csv', index=False)
protests1

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text,fprotets_year
0,2011,Тверская,Андреапольский,28602000,20.4,0.00,0.0,400.0,13664.0,0.0,...,9.4,19.0,124.0,58572.12,13242.0,642.98,13205.0,305086.0,Criticize or denounce,2011
1,2011,Архангельская,Архангельск,11701000,538.2,2.94,2400.0,5900.0,355623.0,-0.7,...,11.1,11.8,3957.0,29587.00,356073.0,417.00,27871.7,29445.0,Criticize or denounce,2011
2,2011,Башкортостан,Аургазинский,80605000,2.1,12.00,0.0,30100.0,36882.0,0.0,...,13.0,16.0,473.0,2239.00,36555.0,426.00,12294.5,201403.0,Criticize or denounce,2011
3,2011,Кабардино-Балкарская,Баксанский,83610000,62.6,0.00,1500.0,81050.0,61054.0,10.4,...,20.1,9.7,1227.0,1325.82,61147.0,225.00,9631.4,75350.0,Criticize or denounce,2011
4,2011,Башкортостан,Благовещенск,80615101,0.0,0.00,0.0,4000.0,34286.0,0.0,...,0.0,0.0,499.0,0.00,0.0,0.00,0.0,6466.0,Demonstrate or rally,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2013,Волгоградская,Серафимовичский,18650000,35.3,80.90,200.0,1800.0,24874.0,-2.6,...,10.6,13.2,261.0,2904.97,24677.0,0.00,14716.9,432352.9,Criticize or denounce,2013
150,2013,Иркутская,Слюдянский,25634000,53.4,44.83,73.0,900.0,40383.0,4.0,...,20.0,15.0,784.0,40387.00,40286.0,0.00,29371.7,630111.0,Criticize or denounce,2013
151,2013,Костромская,Солигаличский,34640000,0.0,28.95,50.0,349.0,9689.0,-3.1,...,14.9,18.0,143.0,4874.00,9611.0,0.00,18141.7,305492.0,Demonstrate or rally,2013
152,2013,Тверская,Торжокский,28654000,4.8,0.00,0.0,4260.0,23071.0,-4.0,...,12.0,16.0,269.0,5141.00,23119.0,0.00,18730.3,312810.0,Criticize or denounce,2013


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [21]:
lol = pd.read_csv("протесты_2012t+.csv") 
sobytia = pd.read_csv("общие_данные_2012t+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2012t+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623222,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623223,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623224,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623225,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [22]:
lol = pd.read_csv("протесты2012_2012t+.csv") 
sobytia = pd.read_csv("общие_данные_2012t+.csv")

df_2011 = sobytia[sobytia['year'] == 2010]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2011_2012t+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2010,Карачаево-Черкесская,Абазинский,91601000,0.00,0.00,0.0,3800.0,13280.0,0.0,...,-1356.2,11.1,9.1,191.0,1447.9,17175.0,230.00,10943.0,30019.0,NaN
1,2010,Удмуртская,Алнашский,94602000,22.40,0.00,0.0,660.0,20718.0,0.0,...,3398.0,18.9,16.4,387.0,6349.1,20428.0,360.00,8804.1,89600.0,Investigate
2,2010,Мурманская,Апатиты,47705000,1.40,0.00,300.0,500.0,60003.0,-4.2,...,52509.0,10.9,15.1,655.0,0.0,59826.0,510.80,25817.6,247136.0,NaN
3,2010,Башкортостан,Архангельский,80603000,6.20,0.00,0.0,0.0,19829.0,0.0,...,8468.0,14.6,17.3,271.0,1610.0,18503.0,483.00,11085.9,242145.0,NaN
4,2010,Башкортостан,Архангельский,80603410,0.00,0.00,0.0,0.0,0.0,0.0,...,-292.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6347.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2010,Калмыкия,Элиста,85701000,51.30,0.00,0.0,2963.0,106966.0,0.0,...,-86985.4,16.2,10.1,1775.0,11329.0,109647.0,323.00,14293.4,39657.0,Reject
100,2010,Сахалинская,Южно-Сахалинск,64701000,171.00,1.03,0.0,4700.0,0.0,0.0,...,-427701.0,12.1,11.4,2286.0,18556.0,189069.0,389.08,43408.3,89827.0,Consult
101,2010,Саха,Якутск,98701000,311.76,0.00,1800.0,1300.0,278691.0,0.0,...,1380480.0,19.0,8.3,5426.0,0.0,285106.0,319.00,32882.1,355334.0,"Arrest, detain, or charge with legal action"
102,2010,Башкортостан,Янаульский,80659000,10.70,0.00,200.0,5000.0,48916.0,0.0,...,2967.0,14.3,14.9,690.0,564.0,48213.0,432.00,17995.5,209381.0,Make statement


Создание регрессии

In [23]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2012_2012t+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2012t+.csv")
df_events_full = pd.read_csv("общие_данные_2012t+.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',
            # 'Квартиры',  
            
            # 'Население', 
            # 'Прибыль',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)

exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     'Советский': ['Будённовский', 'Степновский', 'Кировский', 'Георгиевский', 'Новоселицкий'], 
     'Ульяновский': ['Ульяновск', 'Чердаклинский', 'Новоульяновск', 'Сенгилеевский', 'Теренгульский', 'Майнский', 'Цильнинский', 'Тетюшский'],
     "Владивостокский": ["Хасанский", "Надеждинский", "Артемовский", "Шкотовский", "Большой Камень", "Фокино"],
    "Воротынский": ["Пильнинский", "Спасский", "Лысковский", "Воскресенский", "Юринский", "Горномарийский", "Ядринский"],
    "Джейрах": ["Итум-Калинский", "Ачхой-Мартановский", "Сунженский", "Пригородный", "Владикавказ"],
    "Кизилюртовский": ["Кумторкалинский", "Хунзахский", "Буйнакский", "Казбековский", "Хасавюртовский",
                       "Кизилюрт", "Цумадинский", "Бабаюртовский", "Ахвахский"],
    "Кичменгско-Городецкий": ["Нюксенский", "Бабушкинский", "Никольский", "Вохомский", "Опаринский", "Подосиновский", "Великоустюгский"],
    "Красногвардейский": ["Усть-Лабинский", "Шовгеновский", "Белореченский", "Теучежский", "Краснодар", "Динский"],
    "Красноярск": ["Березовский", "Дивногорск", "Емельяновский"],
    "Нестеровский": ["Краснознаменский", "Гусевский", "Озерский"],
    "Сегежский": ["Беломорский", "Муезерский", "Медвежьегорский", "Онежский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"],
    "Хасавюртовский": ["Хасавюрт", "Новолакский", "Ахвахский", "Гудермесский", "Шелковский",
                       "Бабаюртовский", "Хунзахский", "Кизилюртовский", "Кизилюрт", "Казбековский"],
    "Чайковский": ["Еловский", "Куединский", "Янаульский", "Нефтекамск", "Камбарский",
                   "Сарапульский", "Завьяловский", "Воткинский"],
    "Шовгеновский": ["Курганинский", "Кошехабльский", "Гиагинский", "Белореченский", "Красногвардейский", "Усть-Лабинский"],
    "Южно-Сахалинск": ["Корсаковский", "Анивский", "Холмский", "Долинский"]
}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2012t+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,...,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,mahalanobis_distance
0,Тверская,Андреапольский,28602000,2011,20.4,0.00,0.0,400.0,953.00,5.30,...,14.1,0.00,0.0,185.0,912.00,0.00,25531.3,3.3,167105.00,0.780967
1,Тверская,Селижаровский,28650000,2011,24.2,0.00,100.0,1320.0,1328.00,0.00,...,2.1,0.00,0.0,0.0,1318.00,0.00,6970.0,4.7,163299.00,0.540647
2,Тверская,Тверь,28701000,2011,76.1,0.00,0.0,300.0,692.30,34.90,...,64.4,90.00,2400.0,8373.0,580.00,40.00,-58584.0,5.0,133976.00,4.165207
3,Тверская,Торопецкий,28655000,2012,29.1,0.00,0.0,1400.0,702.00,0.00,...,114.5,0.00,200.0,595.0,582.00,0.00,13916.0,3.0,281587.00,0.410849
4,Тверская,Торжокский,28654000,2013,4.8,0.00,0.0,4260.0,684.00,37.30,...,5.9,0.00,0.0,0.0,482.20,31.70,43006.0,0.0,5852.00,0.869374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Ненецкий,Заполярный,11811000,2013,16.6,0.00,120.0,1339.0,1084.00,25.40,...,0.0,0.00,0.0,939.0,0.00,0.00,-72161.0,0.0,889.86,29.570765
136,Ростовская,Кашарский,60624000,2013,13.8,46.42,0.0,500.0,528.00,0.00,...,0.0,75.23,0.0,0.0,543.60,0.00,-3010.0,0.8,469456.00,0.242880
137,Ханты-Мансийский,Мегион,71873000,2013,93.3,0.00,0.0,0.0,1433.40,20.75,...,12.3,94.93,0.0,0.0,993.50,18.70,-363931.0,0.0,16890.00,1.080182
138,Ханты-Мансийский,Нягань,71879000,2013,211.7,88.52,0.0,1600.0,1111.43,19.75,...,6.6,0.00,0.0,0.0,998.89,18.01,-76391.0,0.0,4515.23,0.328358


In [24]:
df_2012 = pd.read_csv('прототип_данных_2012t+.csv')
df_2011 = pd.read_csv('без_протестов2011_2012t+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2012t+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Западнодвинский,28616000,114.5,0.00,200.0,595.0,2012
1,Торжокский,28654000,4.8,0.00,0.0,4260.0,Торжок,28750000,5.9,0.00,0.0,0.0,2013
2,Холмогорский,11656000,90.0,0.00,620.0,1601.0,Верхнетоемский,11608000,87.6,0.00,0.0,3350.0,2012
3,Аургазинский,80605000,2.1,12.00,0.0,30100.0,Бакалинский,80607000,9.1,14.30,500.0,2000.0,2011
4,Янаульский,80659000,20.0,14.20,0.0,5800.0,Ишимбайский,80631000,15.8,67.20,0.0,7700.0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Варгашинский,37606000,33.5,0.00,0.0,3700.0,Притобольный,37630000,8.4,93.49,0.0,100.0,2013
87,Кашарский,60624000,13.8,46.42,0.0,500.0,Заветинский,60617000,0.0,75.23,0.0,0.0,2013
88,Мегион,71873000,93.3,0.00,0.0,0.0,Радужный,71877000,12.3,94.93,0.0,0.0,2013
89,Нягань,71879000,211.7,88.52,0.0,1600.0,Покачи,71884000,6.6,0.00,0.0,0.0,2013


In [25]:
df = pd.read_csv('общие_данные_2012t+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2012t+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2012t+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008,Торопецкий,28655000,24.0,0.00,300.0,500.0,Тверская,Западнодвинский,28616000.0,28.0,0.00,600.0,800.0,Тверская,NaN
1,2009,Торопецкий,28655000,29.0,0.00,0.0,0.0,Тверская,Западнодвинский,28616000.0,113.3,0.00,300.0,400.0,Тверская,NaN
2,2010,Торопецкий,28655000,29.1,0.00,0.0,0.0,Тверская,Западнодвинский,28616000.0,114.3,0.00,300.0,595.0,Тверская,NaN
3,2011,Торопецкий,28655000,29.1,0.00,0.0,500.0,Тверская,Западнодвинский,28616000.0,114.5,0.00,200.0,595.0,Тверская,2012.0
4,2012,Торопецкий,28655000,29.1,0.00,0.0,1400.0,Тверская,Западнодвинский,28616000.0,104.4,0.00,100.0,200.0,Тверская,2012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,2016,Солигаличский,34640000,0.0,38.09,0.0,1.0,Костромская,Вохомский,34606000.0,0.0,13.47,0.0,7600.0,Костромская,NaN
1161,2017,Солигаличский,34640000,0.0,38.52,0.0,351.0,Костромская,Вохомский,34606000.0,0.0,13.97,0.0,280.0,Костромская,NaN
1162,2018,Солигаличский,34640000,0.0,38.52,0.0,122.0,Костромская,Вохомский,34606000.0,0.0,14.18,0.0,7000.0,Костромская,NaN
1163,2019,Солигаличский,34640000,0.0,38.71,0.0,545.0,Костромская,Вохомский,34606000.0,0.0,14.21,0.0,700.0,Костромская,NaN
